# Model Training and Decision Making
## Credit Risk Analysis

### Objetivo
Entrenar y evaluar un modelo de clasificación binaria que permita apoyar la toma de
decisiones en la aprobación de créditos, minimizando el riesgo de incumplimiento.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    confusion_matrix,
    classification_report,
    roc_curve
)

In [ ]:
df = pd.read_csv("../data/processed/maria_EDA.csv")
df.head()

In [ ]:
X = df.drop(columns=["target"])
y = df["target"]

X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

La estratificación garantiza que la proporción de clientes riesgosos y no riesgosos se
mantenga constante en los conjuntos de entrenamiento y prueba.

In [ ]:
model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced"
)

model.fit(X_train, y_train)

In [ ]:
y_proba = model.predict_proba(X_test)[:,1]
auc = roc_auc_score(y_test, y_proba)
auc

El valor del AUC indica la capacidad del modelo para discriminar entre clientes con y sin
riesgo de incumplimiento.

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_proba)

plt.figure()
plt.plot(fpr, tpr)
plt.plot([0,1], [0,1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()

In [ ]:
y_pred_default = (y_proba >= 0.5).astype(int)
confusion_matrix(y_test, y_pred_default)

print(classification_report(y_test, y_pred_default))

In [ ]:
threshold = 0.35
y_pred_custom = (y_proba >= threshold).astype(int)

confusion_matrix(y_test, y_pred_custom)

print(classification_report(y_test, y_pred_custom))

In [ ]:
La modificación del umbral de decisión permite analizar distintos escenarios de riesgo,
equilibrando la tasa de aprobación de créditos y el riesgo de incumplimiento.

In [ ]:
decision_df = X_test.copy()
decision_df["prob_default"] = y_proba
decision_df["decision"] = decision_df["prob_default"].apply(
    lambda x: "RECHAZAR" if x >= threshold else "APROBAR"
)

decision_df.head()

In [ ]:
importance = pd.DataFrame({
    "Variable": X.columns,
    "Coeficiente": model.coef_[0]
}).sort_values("Coeficiente", ascending=False)

importance

Las variables con coeficientes positivos incrementan la probabilidad de incumplimiento,
mientras que coeficientes negativos están asociados a menor riesgo crediticio.

## Conclusiones

- El modelo de regresión logística presentó un desempeño adecuado para la predicción del
riesgo crediticio.
- El uso de un umbral de decisión personalizado permite adaptar la política de aprobación
según el apetito de riesgo.
- El modelo constituye una herramienta válida de apoyo a la toma de decisiones en
contextos académicos.